<a href="https://colab.research.google.com/github/wangjb/cwb_open_data/blob/master/rt_precip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 讀取CWB即時雨量資料

In [60]:
import urllib
# query string to CWB open data service
url='https://opendata.cwb.gov.tw/'
sub_path='fileapi/v1/opendataapi/'
# instant 10 min precip
datacode='O-A0002-001'
auth=None # your auth code
datalimit=''
dataoffset=''
dataformat='JSON'
stationID=''
datastatus=''

# create query string
url_path=url+sub_path+datacode
query_fields=['Authorization','limit','offset','format','stationID','status']
query_data=(auth,datalimit,dataoffset,dataformat,stationID,datastatus)
query_string=dict(zip(query_fields,query_data))
print(url_path)

https://opendata.cwb.gov.tw/fileapi/v1/opendataapi/O-A0002-001


In [70]:
import requests
r = requests.get(url_path,params=query_string,stream=True)
jsondata = r.json()

In [81]:
from datetime import datetime
data = [ [ site['stationId'],
        site['locationName'],
        float(site['lat']),
        float(site['lon']),
        site['time']['obsTime'],
        [float(ele['elementValue']['value']) for ele in site['weatherElement'] ] ]  for site in jsondata['cwbopendata']['location'] ]

In [231]:
jsondata['cwbopendata']['location'][0].keys()

dict_keys(['lat', 'lon', 'locationName', 'stationId', 'time', 'weatherElement', 'parameter'])

In [232]:
data = pd.DataFrame(jsondata['cwbopendata']['location'])

# process obstime
data['obstime'] = pd.DataFrame(data['time'].to_list())

# process weatherElement
weatherElement = pd.DataFrame(data['weatherElement'].to_list())
weatherElement.columns = [ x['elementName'] for x in weatherElement.iloc[0] ]
weatherElement = weatherElement.applymap(lambda x: float(x['elementValue']['value']))

# process parameter
parameter = pd.DataFrame(data['parameter'].to_list())
parameter.columns = [ x['parameterName'] for x in parameter.iloc[0] ]
parameter = parameter.applymap(lambda x: x['parameterValue'])

# merge data
data = data.merge(weatherElement,left_index=True,right_index=True)
data = data.merge(parameter,left_index=True,right_index=True)
data = data.drop(['time','weatherElement','parameter'],axis=1)

In [233]:
data

,lat,lon,locationName,stationId,obstime,ELEV,RAIN,MIN_10,HOUR_3,HOUR_6,HOUR_12,HOUR_24,NOW,latest_2days,latest_3days,CITY,CITY_SN,TOWN,TOWN_SN,ATTRIBUTE
0,23.9637,120.8369,九份二山,C1I230,2020-08-07T13:30:00+08:00,837.0,-998.0,-998.0,-998.0,-998.0,0.0,1.0,0.0,1.0,2.5,南投縣,13,國姓鄉,139,局屬無人測站
1,25.1351,121.7323,基隆,466940,2020-08-07T13:30:00+08:00,26.7,-998.0,-998.0,-998.0,-998.0,0.0,0.0,0.0,0.0,0.5,基隆市,03,仁愛區,015,中央氣象局
2,25.1667,121.4407,淡水,466900,2020-08-07T13:30:00+08:00,19.0,-998.0,-998.0,-998.0,-998.0,0.0,-999.0,0.0,0.0,0.0,新北市,06,淡水區,004,中央氣象局
3,24.9994,121.4338,板橋,466880,2020-08-07T13:30:00+08:00,11.0,-998.0,-998.0,-998.0,-998.0,0.0,0.0,0.0,0.0,0.5,新北市,06,板橋區,041,中央氣象局
4,25.1639,121.5363,陽明山,466930,2020-08-07T13:30:00+08:00,607.1,-998.0,-998.0,-998.0,-998.0,0.0,0.0,0.0,0.0,0.0,臺北市,01,北投區,006,中央氣象局
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001,24.2285,120.5809,國三S178K,CAF080,2020-08-07T13:30:00+08:00,208.0,-998.0,-998.0,-998.0,-998.0,0.0,0.0,0.0,0.0,0.0,臺中市,02,沙鹿區,120,局屬合作測站
1002,24.0942,120.5951,國三S202K,CAG060,2020-08-07T13:30:00+08:00,48.0,-998.0,-998.0,-998.0,-998.0,0.0,0.0,0.0,0.0,0.0,彰化縣,14,彰化市,142,局屬合作測站
1003,24.0086,120.6451,國三S217K,CAH020,2020-08-07T13:30:00+08:00,93.0,-998.0,-998.0,-998.0,-998.0,0.0,0.0,0.0,0.0,0.5,南投縣,13,草屯鎮,152,局屬合作測站
1004,24.2982,120.6350,國四E5K,CAF090,2020-08-07T13:30:00+08:00,130.0,-998.0,-998.0,-998.0,-998.0,0.0,0.0,0.0,0.0,0.0,臺中市,02,神岡區,114,局屬合作測站
